In [75]:
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import numpy as np
import collections as col
import scipy.sparse as sp
from scipy.sparse import lil_matrix


In [76]:
data_test = pd.read_csv("./arxiv_articles.csv", sep="|",usecols=[1,3,4],nrows=20)

In [77]:
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))
#fonctions appliquées sur chaque document pour nettoyage
b = lambda new : " ".join([ps.stem(i) for i in re.sub("[^a-zA-Z]", " " ,new.lower()).split() if i not in stop_words]).split()
#a = lambda x : col.Counter(re.findall('\w+',x))

In [78]:
data_test['cleaned']=data_test['summary'].apply(b)
#data_test['dict']=data_test['cleaned'].apply(a)

In [79]:
def calcul_tf(d):
    term_counts = pd.Series(d).value_counts()
    max_mot = max(term_counts)
    return 0.5 + 0.5 * (term_counts / max_mot)
def compute_idf(D):
    N = len(D)
    all_terms = pd.concat(D)
    nt = all_terms.index.value_counts() # The number of documents containing the term "t"
    return np.log(N / nt)
def compute_tf_idf_document(tf_document, idf):
    return tf_document * np.array([idf[i] for i in tf_document.index])
#calcul le tf idf pour 1 document
def compute_tf_idf_corpus(D):
    term_freq = [calcul_tf(d) for d in D]
    idf = compute_idf(term_freq)
    return [compute_tf_idf_document(d, idf) for d in term_freq]
#prend en entree une colonne panda series et return un 

In [123]:
tf_idf = compute_tf_idf_corpus(data_test.loc[:5, "cleaned"])
#liste avec comme indice le numéro du document et pour chaque document un panda series 
#avec mot comme index et tf_idf pour le mot comme values
#array de nos mots
all_mot = pd.concat(tf_idf).index #attention il n'additionne pas si deux fois le même mot
print((tf_idf.append(self=tf_idf,))

AttributeError: 'list' object has no attribute 'toappend'

In [90]:
for i in len(all_mot):
    if data.loc[:5, all_mot[i]].notnull() in  
    mat[:,i]=

(92,)


In [120]:
data=pd.DataFrame(tf_idf)
print(data.loc[:5, 'group'])
c = lambda x : x if x not 'NaN'

SyntaxError: invalid syntax (<ipython-input-120-eff3d2bdcf32>, line 3)

In [55]:
idf = compute_idf(term_freq)